In [47]:
import pandas as pd
import os
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

In [48]:
working_dir = os.getcwd()
rel_path = r"project_resources\chembl_molekuly.csv"
molekuly_csv_abs_file_path = os.path.join(working_dir, rel_path)
#loads csv file from directory

In [49]:
#dir for useful stuff for the actual essay
rel_path = r"project_results\graphs"
project_results_graphs = os.path.join(working_dir, rel_path)

In [50]:
df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")
#df.head()

C:\Users\Lukas\AppData\Local\Temp\ipykernel_15308\3345623085.py:1: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")


In [51]:
df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")
statistika = df["Standard Type"].value_counts()
#print(statistika)

C:\Users\Lukas\AppData\Local\Temp\ipykernel_15308\4010148655.py:1: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")


In [52]:
result = df[(df["Standard Type"] == "t1/2") | (df["Standard Type"] == "T1/2")]
t_half_rel_path = r"project_resources\t_half.csv"
t_half_abs_path = os.path.join(working_dir, t_half_rel_path)
result.to_csv(t_half_abs_path, index=False)
#creates "t_half.csv" in current dir
t_half = pd.read_csv(t_half_abs_path)
#and then loads it

In [53]:
client = new_client
list_smiles = []
list_chembl_id = t_half["Molecule ChEMBL ID"]
for chembl_id in list_chembl_id:
    molecule = client.molecule
    compound = molecule.filter(chembl_id=chembl_id)[0]
    list_smiles.append(compound['molecule_structures']["canonical_smiles"])
#print(list_smiles)

In [54]:
#takes a list of smiles strings,output is a corresponding Morgan fingerprint as a list
list_fingerprint = []
for smi in list_smiles:
    mol = Chem.MolFromSmiles(smi)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, useChirality=True, radius=2, nBits = 124)
    vector = np.array(fingerprint)
    list_fingerprint.append(vector)
    #print(list_fingerprint)

In [57]:
#data sets filtering for ML
t_half = pd.read_csv(t_half_abs_path)

t_half = t_half['Standard Value']
t_half_float = t_half.astype(float)
t_filtered = t_half_float[t_half>0]

array_fingerprint = np.array(list_fingerprint)
array_fingerprint_filtered = array_fingerprint[t_half>0]

In [56]:
list_fingerprint[t_half>0]

TypeError: list indices must be integers or slices, not Series

In [58]:
# x su len fingerprint ako jeden numpy array,
# y su len t_half hodnoty ako jeden numpy array, pouzit len molekuly kde existuje aj X aj y hodnota
x = array_fingerprint_filtered
y = t_filtered
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(t_filtered.shape, array_fingerprint_filtered.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(70,) (70, 124) (56, 124) (14, 124) (56,) (14,)


In [59]:
def model_choice(ml_type):
    
    if ml_type == 'linear':
        reg = linear_model.ElasticNet(alpha=0.001, l1_ratio=1)
        plot_title = "Linear"
        
    if ml_type == 'KRR':
        reg = KernelRidge(alpha=0.1)
        plot_title = "Kernel ridge regressor"
        
    if ml_type == 'RF':
        reg = RandomForestRegressor(max_depth=None, random_state=0)
        plot_title = "Random forest regressor"
        
    if ml_type == 'GB':
        reg = GradientBoostingRegressor(random_state=1, loss='absolute_error', max_depth=5, n_estimators=1000)
        plot_title = "Gradient boosting regressor"
        
    if ml_type == 'ANN':
        reg = MLPRegressor(random_state=1, max_iter=5000, learning_rate_init=0.001, alpha=0, hidden_layer_sizes=[10,10])
        plot_title = "Limited-memory BFGS"
    print(f"using ML type {ml_type}")
    
    return reg, plot_title

In [60]:
def mach_learn(x_train, x_test, y_train, y_test, reg, plot_title):
    # výcvik strojoveho modelu
    reg = reg.fit(x_train, y_train)
    # predict training set
    y_train_predict = reg.predict(x_train)
    # predict test set
    y_test_predict = reg.predict(x_test)
    #error train je pre info, test je dolezita
    train_error = np.mean(np.abs(y_train_predict-y_train))
    test_error = np.mean(np.abs(y_test_predict-y_test))
    print(f"MAE train: {train_error:.4f}")
    print(f"MAE test: {test_error:.4f}")

    # idealne data na diagonale
    diag = np.linspace(-1, 3.5)

    #plot = plt.scatter(y_test,y_test_predict,edgecolors=None,c='b',alpha=0.2)
    #plt.plot(diag, diag, linestyle='dotted')
    #plt.xlim(-1, 3.5)
    #plt.ylim(-1, 3.5)
    #plt.ylabel('$t_{1/2}$ (predicted)')
    #plt.xlabel('$t_{1/2}$ (real)')
    #plt.suptitle(f"{plot_title}")
    #plt.savefig(os.path.join(project_results_graphs, plot_title))
    #plt.show()

In [61]:
# opakovat pre rozne strojove metody, porovnan chyby
#to display graphs, un-comment all rows in previous cell starting with plot or plt
type_ml_use = 'linear'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type linear
MAE train: 0.2470
MAE test: 0.7197


C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.672e-02, tolerance: 6.966e-03
  model = cd_fast.enet_coordinate_descent(


In [62]:
type_ml_use = 'KRR'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type KRR
MAE train: 0.2274
MAE test: 0.6352


In [63]:
type_ml_use = 'RF'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type RF
MAE train: 0.3392
MAE test: 0.5554


In [64]:
type_ml_use = 'GB'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type GB
MAE train: 0.2144
MAE test: 0.3618


In [65]:
type_ml_use = 'ANN'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type ANN
MAE train: 0.2984
MAE test: 0.4429


In [66]:
def param_tuning(type_ml_use, reg):
    if type_ml_use == 'linear':
        param_grid = {
            'fit_intercept': [True], 
            'alpha': [1e-5, 1e-4, 1e-3, 1e-2], 
            'l1_ratio': [0, 0.1, 0.5, 0.9, 1]
        }
        reg = linear_model.ElasticNet()

    if type_ml_use=='KRR':
        param_grid= {
            "alpha": np.logspace(-4, 1, 20),
            "gamma": np.logspace(-14, 0, 20), 
            "kernel": ['linear', 'laplacian', 'rbf']
        }
        reg = KernelRidge()

    if type_ml_use=='GB':
        param_grid = {
            'n_estimators': [20, 200, 2000], 
            'learning_rate': [0.02, 0.05], 
            'max_depth': [1, 2, 3, 5], 
            'min_samples_leaf': [5, 10, 20], 
            'min_samples_split': [2, 5]
        }
        reg = GradientBoostingRegressor()

    if type_ml_use=='RF':
        param_grid = {
            'max_depth': [None, 2, 3, 5, 10], 
            'max_features': ['auto','sqrt', 'log2'], 
            'n_estimators': [10, 20, 50, 100, 200],
        }
        reg = RandomForestRegressor()

    if type_ml_use=='ANN':
        param_grid = {
            'learning_rate_init': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05], 
            'hidden_layer_sizes': [[5], [10], [20], [50], [5]*2, [10]*2, [20]*2, [50]*2, [5]*3, [10]*3]
        }
        reg = MLPRegressor()

    grid = GridSearchCV(reg, param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=1), verbose=0)
    grid.fit(x_train, y_train)
    best_reg = grid.best_estimator_
    print(type_ml_use, str(grid.best_params_))
    # retrain on best hyperparameters
    best_reg.fit(x_train, y_train)
    # vypocitat novy MAE train, test po hyperparameter trainink
    y_train_predict = best_reg.predict(x_train)
    y_test_predict = best_reg.predict(x_test)
    train_error = np.mean(np.abs(y_train_predict-y_train))
    test_error = np.mean(np.abs(y_test_predict-y_test))
    print(f"MAE train: {train_error:.4f}")
    print(f"MAE test: {test_error:.4f}")

In [67]:
type_ml_use = 'linear'
reg, plot_title = model_choice(type_ml_use)
param_tuning(type_ml_use, reg)

using ML type linear


C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.994e+00, tolerance: 6.792e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.991e+00, tolerance: 3.599e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemente

linear {'alpha': 0.01, 'fit_intercept': True, 'l1_ratio': 1}
MAE train: 0.3531
MAE test: 0.4430


In [68]:
type_ml_use = 'KRR'
reg, plot_title = model_choice(type_ml_use)
param_tuning(type_ml_use, reg)

using ML type KRR
KRR {'alpha': 10.0, 'gamma': 0.03359818286283774, 'kernel': 'laplacian'}
MAE train: 0.4568
MAE test: 0.3940


In [69]:
type_ml_use = 'RF'
reg, plot_title = model_choice(type_ml_use)
param_tuning(type_ml_use, reg)

using ML type RF


C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the defa

RF {'max_depth': 2, 'max_features': 'log2', 'n_estimators': 200}
MAE train: 0.4877
MAE test: 0.5010


In [70]:
type_ml_use = 'GB'
reg, plot_title = model_choice(type_ml_use)
param_tuning(type_ml_use, reg)

using ML type GB
GB {'learning_rate': 0.05, 'max_depth': 1, 'min_samples_leaf': 20, 'min_samples_split': 2, 'n_estimators': 20}
MAE train: 0.4758
MAE test: 0.5178


In [71]:
type_ml_use = 'ANN'
reg, plot_title = model_choice(type_ml_use)
param_tuning(type_ml_use, reg)

using ML type ANN


C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lukas\anaconda3\envs\soc\Li

ANN {'hidden_layer_sizes': [10], 'learning_rate_init': 0.05}
MAE train: 0.4880
MAE test: 0.3725


In [76]:
def to_rdkit_fingerprint(fps):
    rdkit_fingerprints = []
    for prnt in fps:
        rdkit_fingerprints.append(Chem.RDKFingerprint(Chem.MolFromSmiles(prnt)))
    return rdkit_fingerprints

In [77]:
fps = to_rdkit_fingerprint(list_smiles)
fp2s = to_rdkit_fingerprint(list_smiles)
tanimoto_similarities = []
for x in fps:
    fpsx = []
    for y in fp2s:
        fpsx.append(DataStructs.TanimotoSimilarity(x,y))
    smallest_tanimoto = min(fpsx)
    tanimoto_similarities.append(smallest_tanimoto)
print(tanimoto_similarities)

[0.1920583468395462, 0.26339004410838057, 0.24747937671860679, 0.2791346824842987, 0.27364864864864863, 0.2754145638067772, 0.23171733771569433, 0.2754145638067772, 0.24851190476190477, 0.2349869451697128, 0.21160409556313994, 0.27304469273743015, 0.26356589147286824, 0.2733727810650888, 0.26372924648786716, 0.2689075630252101, 0.26789107029765674, 0.2624203821656051, 0.2815395580898076, 0.27528901734104044, 0.26308813783962887, 0.26857142857142857, 0.2786103542234332, 0.27613293051359517, 0.2725587144622991, 0.21553090332805072, 0.2523364485981308, 0.2637992831541219, 0.27383863080684595, 0.296957671957672, 0.27090779127948533, 0.2733727810650888, 0.27400611620795107, 0.26841018582243636, 0.2701801200800534, 0.20625465376023827, 0.27667493796526055, 0.27383863080684595, 0.2692857142857143, 0.26711409395973157, 0.2682926829268293, 0.1920583468395462, 0.27518593644354294, 0.2791346824842987, 0.20419847328244276, 0.27062937062937065, 0.21160409556313994, 0.27530364372469635, 0.2736992159

In [74]:
len(tanimoto_similarities)

74

In [75]:
afps = ['COc1ccc2[nH]cc(CCNC(C)=O)c2c1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)[C@@H](CN2Cc3c(F)cccc3C2=O)CN1', 'O=c1[nH]c2ccccc2n1C1CCN(CCCC(c2ccc(F)cc2)c2ccc(F)cc2)CC1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc([C@@H](O)c4ccc(F)cc4)cc32)[C@@H](CN2[C@H](C)COC[C@H]2C)CN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)[C@@H](CN2Cc3ccccc3C2=O)CN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)[C@@H](CN2[C@H](C)COC[C@@H]2C)CN1', 'CCC(=O)NCC[C@@H]1CCc2ccc3c(c21)CCO3', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)[C@@H](CN2[C@H](C)COC[C@H]2C)CN1', 'COc1cc(N)c(Cl)cc1C(=O)NC1CCN(CCCOc2ccc(F)cc2)CC1OC', 'CC(C)O[P@@]1(=O)OC[C@H]2O[C@@H](n3ccc(=O)[nH]c3=O)[C@@]3(CCO3)[C@@H]2O1', 'CCS(=O)(=O)c1ccc2oc(-c3ccc4ccccc4c3)nc2c1', 'CC1=C(C)C(=O)N(C[C@H]2CN[C@H](C)CN2CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)C1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)[C@@H](CN2Cc3ccccc3C2=O)CN1', '[2H]C([2H])(O)C1(C([2H])([2H])O[C@]2(c3ccc(Cl)cc3)c3c(F)cc(C(C)(C)O)cc3C(=O)N2Cc2ccc(Cl)cn2)CC1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)[C@@H](CN2Cc3cc(F)ccc3C2=O)CN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)[C@@H](CN2C[C@@H](C)OCC2=O)CN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)[C@@H](CN2Cc3ccc(C#N)cc3C2=O)CN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)[C@@H](CN2Cc3ccc(F)cc3C2=O)CN1', 'COc1ccc(CCN2CCC(Nc3nc4ccccc4n3Cc3ccc(F)cc3)CC2)cc1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)[C@@H](CN2CCOC[C@H]2C)CN1']
afp2s = ['C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4)cc32)C@@HCN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)C@@HCN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3[nH]c(=O)c(Cc4ccc(F)cc4)cc32)C@@HCN1', 'Cc1ncsc1-c1ccc(C@Hccc(F)c2F)NC(=O)[C@@H]2CC@@HCN2C(=O)C@@HC(C)(C)C)cc1', 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@@]2(OCC1(O)CC1)c1ccc(Cl)cc1', 'O=C(NC1CCN(CCc2c[nH]c3ccccc23)CC1)c1ccccc1', 'O=C(CCCN1CCC(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F)cc1', 'CC(C)(O)c1ccc2c(c1)C(=O)N(Cc1ccc(Cl)cc1)[C@@]2(OCC1(CO)CC1)c1ccc(Cl)cc1', 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@@]2(OCC1(C(N)=O)CC1)c1ccc(Cl)cc1', 'O=C1CCC(N2C(=O)c3cccc(NCCOCCOCCNC(=O)c4ccc5c(c4)nc(Nc4cccc(Cl)c4)c4ccncc45)c3C2=O)C(=O)N1', 'COc1cccc(C@@HNC(=O)C@@HN2Cc3ccc(-c4nc(NC5CCOCC5)ncc4Cl)cc3C2=O)c1', '[2H]C([2H])(O)C1(C([2H])([2H])O[C@]2(c3ccc(Cl)cc3)c3c(F)cc(C(C)(C)O)cc3C(=O)N2Cc2ccc(Cl)cn2)CC1', 'O=C(COc1c(-c2csc(N3CCOCC3)n2)ccc(F)c1F)NCCOCCOCCNc1cccc2c1C(=O)N(C1CCC(=O)NC1=O)C2=O', 'COc1cc(F)cc(C@Hcc3C2=O)C(C)O)c1', 'COc1cc(F)cc(C@@HNC(=O)C@@HN2Cc3ccc(-c4nc(Nc5ccnn5C)ncc4Cl)cc3C2=O)c1', 'CC1(C)O[C@@H]2C[C@H]3[C@@H]4CCC5=CC(=O)C=C[C@]5(C)[C@@]4(F)C@@HC[C@]3(C)[C@]2(C(=O)CO)O1', 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(C#N)cn1)[C@@]2(OCC1(CO)CC1)c1ccc(Cl)cc1', 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@@]2(OCC1(C(N)=O)CC1)c1ccc(Cl)cc1', 'COc1cc(F)cc(C@@HNC(=O)C@@HN2Cc3ccc(-c4ccnc(NC5CCOCC5)n4)cc3C2=O)c1', 'COc1cc(F)cc(C@@HNC(=O)C@@HN2Cc3ccc(-c4nc(Nc5ccnc(OC)c5)ncc4Cl)cc3C2=O)c1']
bfps = []
for i in afps:
    bfps.append(Chem.RDKFingerprint(Chem.MolFromSmiles(i)))
bfp2s = bfps[:10]
bfps = bfps[10:]
#print(bfp2s)
tanimoto_similarities = [DataStructs.TanimotoSimilarity(x,y) for x,y in zip(bfps,bfp2s)]
print(tanimoto_similarities)

[0.2363138686131387, 0.676343421865349, 0.35819975339087545, 0.5645776566757493, 0.7776341305753071, 0.9364207221350078, 0.30933496034167174, 0.6806883365200764, 0.3579676674364896, 0.44324631101021567]
